In [ ]:
import os
import pygmt
import pandas as pd
import numpy as np

In [ ]:
# Creating the path for both the folders
datadir = "./"
path_plots = os.path.join(datadir, "Plots")
path_ev = os.path.join(datadir, "Events")
path_st = os.path.join(datadir, "Stations")

try: os.mkdir(path_plots)
except: print("The directory already exists")

In [ ]:
# Define your study site
minlon, maxlon = 90, 104
minlat, maxlat = 16, 30
lonc = (minlon + maxlon) / 2
latc = (minlat + maxlat) / 2

# Define stations provider
provider1 = "IRIS"
provider2 = "GFZ"

# Define etopo data file
topo_data = '@earth_relief_30s' #30 arc second global relief (SRTM15+V2.1 @ 1.0 km)

In [ ]:
# Defining the catalog duration
for i in range(19):
    year1 = 2000 + i
    year2 = 2001 + i

    print(f"For the Time-period: {year1}-{year2}")
    
    fig = pygmt.Figure()
    pygmt.config(MAP_FRAME_TYPE="fancy",MAP_DEGREE_SYMBOL="degree",MAP_FRAME_WIDTH="0.08c",
        MAP_FRAME_PEN="0.1p",MAP_TICK_PEN_PRIMARY="0.1p",MAP_TICK_LENGTH_PRIMARY="0.1c")
    pygmt.config(FORMAT_GEO_MAP="dddD")
    pygmt.config(FONT_LABEL="8p,4",MAP_LABEL_OFFSET="0.1c",FONT_ANNOT_PRIMARY="8p,4",
        FONT_TITLE="10p,4",MAP_TITLE_OFFSET="0.1c")
    
    with fig.subplot(
        nrows=1, ncols=2, figsize = ("8i", "8i"),
        autolabel = False, margins=["0.2c", "0.0c"],
    ):

        fig.grdimage(region=[minlon, maxlon, minlat, maxlat], projection='M?', 
            grid=topo_data, cmap="elevation", shading=True,
            frame=['xa4f2+l"longitude"', 'ya4f2+l"latitude"', 'WsNe'], panel=[0,0]
        )
        fig.coast(frame=False, shorelines=True, borders='1/1p,black')

        # Plot events
        df = pd.read_csv(f"{path_ev}/Catalog_{year1}-{year2}", sep=' ', usecols=[0,1,2,3,4])
        df.columns = ["date","lat","lon","dep","mag"]
        pygmt.makecpt(cmap="seis", series=[0,200], background='o')
        fig.plot(x=df["lon"], y=df["lat"], color=df["dep"], cmap=True, style="c0.2c")
        fig.colorbar(position="JMB+o-1.5c/0.3c+w6c/0.15c+h",frame=['x50','y+l"Earthquake Depth (km)"'])

        fig.basemap(region="g", projection=f"R{lonc}/{latc}?", frame=['xa120f60+l"longitude"', 'ya30f15+l"latitude"', 'WsNe'], panel=[0,1])
        fig.coast(frame=['xa120f60+l"longitude"', 'ya30f15+l"latitude"', 'WsNe'], land="snow", water="lightblue")
        
        # Study area
        fig.plot(
            x=[minlon, minlon, maxlon, maxlon, minlon],
            y=[minlat, maxlat, maxlat, minlat, minlat],
            pen="0.5p,red",
        )
        
        # Seismic stations
        try:
            df = pd.read_csv(f"{path_st}/stations_{provider1}_{year1}-{year2}.txt", sep='|', usecols=[4,5], header=0)
            df.columns=["lat","lon"]
            fig.plot(x=df["lon"],y=df["lat"],style="t0.1c",pen="0.1p,black",no_clip=True,label=f"{provider1} Stations")
        except:
            print(f"{provider1} doesn't have data for this particular region and duration.")  

        try:
            df = pd.read_csv(f"{path_st}/stations_{provider2}_{year1}-{year2}.txt", sep='|', usecols=[4,5], header=0)
            df.columns=["lat","lon"]
            fig.plot(x=df["lon"],y=df["lat"],style="t0.05c",pen="0.1p,red",no_clip=True,label=f"{provider2} Stations")
        except:
            print(f"{provider2} doesn't have data for this particular region and duration.")  
        
        fig.legend(position="jbr+o0.1c/0.1c")

    fig.savefig(f"{path_plots}/plot_{year1}_{year2}.png")
    fig.show()